World Coins: A collection of coin images from 32 different currencies.

In [ ]:
# Seznam knihoven
import os
from tensorflow.keras.applications import ResNet50, EfficientNetB3, MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as preprocess_mobilenet

In [ ]:
# Napojení dat a vytvoření cesty
data_dir_train = "data/coins/data/train/"
data_dir_val = "data/coins/data/validation/"
log_dir = "drive/MyDrivedata/data/logs/"
model_dir = "drive/MyDrivedata/data/models/"

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 211
EPOCHS = 50

In [ ]:
# Vytvoření složek, pokud neexistují
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

Model ResNet50

In [24]:
# ImageDataGenerator s odpovídajícím předzpracováním pro ResNet50
train_datagen_resnet = ImageDataGenerator(
    preprocessing_function=preprocess_resnet,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],  # Lehké změny jasu
    channel_shift_range=20.0) # Malé změny barev

val_datagen_resnet = ImageDataGenerator(preprocessing_function=preprocess_resnet)

# Načtení datasetu
train_generator_resnet = train_datagen_resnet.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator_resnet = val_datagen_resnet.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Nastavení steps_per_epoch
steps_per_epoch = len(train_generator_resnet)
validation_steps = max(len(val_generator_resnet), 1)

Found 6174 images belonging to 211 classes.
Found 844 images belonging to 211 classes.


In [ ]:
# Načtení předtrénovaného modelu ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Přidání nových vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu', kernel_regularizer=l2(1e-5))(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join(model_dir, "best_ResNet50.keras"), save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
log_file = os.path.join(log_dir, "ResNet50.csv")
csv_logger = CSVLogger(log_file, append=True)

# Trénování modelu
history = model.fit(
    train_generator_resnet,
    validation_data=val_generator_resnet,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger]
)

In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:50]:
    layer.trainable = False

# Rekompilace
model.compile(optimizer=Adam(learning_rate=3e-6), loss='categorical_crossentropy', metrics=['accuracy'])

# Další fáze trénování
history_fine = model.fit(
    train_generator_resnet,
    validation_data=val_generator_resnet,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr,csv_logger])

# Uložení modelu
best_model_path = os.path.join(model_dir, "best_ResNet50.keras")
if os.path.exists(best_model_path):
    model.save(best_model_path)

Model EfficientNetB3

In [ ]:
train_datagen_efficientnet = ImageDataGenerator(
    preprocessing_function=preprocess_efficientnet,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.9, 1.1])  # Lehké změny jasu

val_datagen_efficientnet = ImageDataGenerator(preprocessing_function=preprocess_efficientnet)

train_generator_efficientnet = train_datagen_efficientnet.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator_efficientnet = val_datagen_efficientnet.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Nastavení steps_per_epoch
steps_per_epoch = len(train_generator_efficientnet)
validation_steps = max(len(val_generator_efficientnet), 1)

Found 6174 images belonging to 211 classes.
Found 844 images belonging to 211 classes.


In [ ]:
# Načtení předtrénovaného modelu EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zmrazení základního modelu

# Přidání nových vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='swish',kernel_regularizer=l2(1e-5))(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_EfficientNetB3.keras", save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
log_file = os.path.join(log_dir, "EfficientNetB3.csv")
csv_logger = CSVLogger(log_file, append=True)

# Trénování modelu
history = model.fit(
    train_generator_efficientnet,
    validation_data=val_generator_efficientnet,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger]
)

# Uložení modelu
model.save("final_efficientnetb3.keras")


In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Rekompilace s nižší learning rate
model.compile(optimizer=Adam(learning_rate=5e-6), loss='categorical_crossentropy', metrics=['accuracy'])

# Další fáze trénování
history_fine = model.fit(
    train_generator_efficientnet,
    validation_data=val_generator_efficientnet,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger]
)

# Uložení finálního doladěného modelu
model.save(os.path.join(model_dir, "final_tuned_efficientnetb3.keras"))


Model MobileNetV3

In [ ]:
train_datagen_mobilenet = ImageDataGenerator(
    preprocessing_function=preprocess_mobilenet,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.95, 1.05],  # Lehké změny jasu
    channel_shift_range=10.0) # Malé změny barev

val_datagen_mobilenet = ImageDataGenerator(preprocessing_function=preprocess_mobilenet)

train_generator_mobilenet = train_datagen_mobilenet.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator_mobilenet = val_datagen_mobilenet.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Nastavení steps_per_epoch
steps_per_epoch = len(train_generator_mobilenet)
validation_steps = max(len(val_generator_mobilenet), 1)

Found 6174 images belonging to 211 classes.
Found 844 images belonging to 211 classes.


In [ ]:
# Načtení předtrénovaného modelu MobileNetV3
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zmrazení základního modelu

# Přidání vlastních vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='hard_swish')(x)
x = Dropout(0.4)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_MobileNetV3.keras", save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
log_file = os.path.join(log_dir, "MobileNetV3_log.csv")
csv_logger = CSVLogger(log_file, append=True)

# Trénování modelu
history = model.fit(
    train_generator_mobilenet,
    validation_data=val_generator_mobilenet,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger]
)

# Uložení modelu
model.save("final_MobileNetV3.keras")

In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:200]:
    layer.trainable = False

# Rekompilace s nižší learning rate
model.compile(optimizer=Adam(learning_rate=7e-6), loss='categorical_crossentropy', metrics=['accuracy'])

# Další fáze trénování
history_fine = model.fit(
    train_generator_mobilenet,
    validation_data=val_generator_mobilenet,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, csv_logger]
)

# Uložení finálního doladěného modelu
model.save(os.path.join(model_dir, "fine_tuned_mobilenetv3_model.keras"))